#### 前言
* Spark 2.X中堆内存的管理方式称为"统一内存管理", 实现在`UnifiedMemoryManager`. 他是Executor端进行内存管理的方式(不是Driver端),
* 统一内存管理分为堆内内存(On-heap Memory)和堆外内存(Off-heap Memory)两大区域，下面对这两块区域进行详细的说明
 

#### 一. 堆内内存   
默认情况下，Spark 仅仅使用了堆内内存。Executor 端的堆内内存区域大致可以分为以下四大块：
* **Execution**内存：主要用于存放 Shuffle、Join、Sort、Aggregation 等计算过程中的临时数据
* **Storage**内存：主要用于存储 spark 的 cache 数据，例如RDD的缓存、unroll数据；
* 用户内存（**User Memory**）：主要用于存储 RDD 转换操作所需要的数据，例如 RDD 依赖等信息。
* 预留内存（**Reserved Memory**）：系统预留内存，会用来存储Spark内部对象。

整个 Executor 端堆内内存如果用图来表示的话，可以概括如下： <img src="img/spark-onheap.png" width="60%">

其中, 图中的几个参数作如下说明:
* `systemMemory = Runtime.getRuntime.maxMemory`，其实就是通过参数`spark.executor.memory`或`--executor-memory`配置的。
* `reservedMemory`在 Spark 2.2.1 中是写死的，其值等于300MB，这个值是不能修改的
* `usableMemory = systemMemory - reservedMemory`，这个就是 Spark 可用内存；
* `spark.memory.storageFraction=0.5`默认

#### 二. 堆外内存  
&nbsp;&nbsp;&nbsp;&nbsp;Spark 1.6 开始引入了Off-heap memory(详见SPARK-11389)。这种模式不在 JVM 内申请内存，而是调用 Java 的 unsafe 相关 API 进行诸如 C 语言里面的 malloc() 直接向操作系统申请内存，由于这种方式不进过 JVM 内存管理，所以可以避免频繁的 GC，这种内存申请的缺点是必须自己编写内存申请和释放的逻辑。
* 默认情况下，堆外内存是关闭的，我们可以通过`spark.memory.offHeap.enabled`参数启用，并且通过`spark.memory.offHeap.size`设置堆外内存大小，单位为字节。
* 如果堆外内存被启用，那么 Executor 内将同时存在堆内和堆外内存，两者的使用互不影响，这个时候 Executor 中的 Execution 内存是堆内的 Execution 内存和堆外的 Execution 内存之和, 堆外内存只区分`Execution内存和 Storage 内存`, 二者的百分比同样由`spark.memory.storageFraction`指定

#### 三. storage内存和executor内存的动态分配
目前 Execution 内存和 Storage 内存可以互相共享的。也就是说，如果 Execution 内存不足，而 Storage 内存有空闲，那么 Execution 可以从 Storage 中申请空间；反之亦然。但关于不在需要占用的对方内存后, 是否要归还给对方有些差异: 
```
storage被占用的内存不会被exect内存归还的, 但是execution被占用的内存, storage是可以归还给它的
```
具体的实现逻辑如下：
* 程序提交的时候我们都会设定基本的 Execution 内存和 Storage 内存区域（通过 spark.memory.storageFraction 参数设置）；
* 在程序运行时，如果双方的空间都不足时，则存储到硬盘；将内存中的块存储到磁盘的策略是按照 LRU 规则进行的。若己方空间不足而对方空余时，可借用对方的空间;（存储空间不足是指不足以放下一个完整的 Block）
* Execution 内存的空间被对方占用后，可让对方将占用的部分转存到硬盘，然后"归还"借用的空间
* Storage 内存的空间被对方占用后，目前的实现是无法让对方"归还"，因为需要考虑 Shuffle 过程中的很多因素，实现起来较为复杂；而且 Shuffle 过程产生的文件在后面一定会被使用到，而 Cache 在内存的数据不一定在后面使用。

注意，上面说的借用对方的内存需要借用方和被借用方的内存类型都一样，都是堆内内存或者都是堆外内存，不存在堆内内存不够去借用堆外内存的空间。